# Books

## From source (no avg ratings)

In [29]:
import pandas as pd
"""
books_all_cols_df = pd.read_csv("../data/goodreads_library_export.csv")
print("n_books",len(books_all_cols_df))
print("columns",books_all_cols_df.columns)
books_df = books_all_cols_df[[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
})
books_df.head()
"""
all_books_df = pd.DataFrame()
for n in range(6): # starts in 0, but page in 1
  books_df = pd.read_xml("../data/50196372-"+str(n+1)+".rss", xpath=".//item")
  all_books_df = pd.concat([all_books_df, books_df])
print("n_books",len(books_all_cols_df))
print("columns",list(books_all_cols_df.columns))
books_df = books_all_cols_df[books_all_cols_df["Exclusive Shelf"]!="missclick"][[
  "Author l-f", "Title", "Read Count", "Average Rating", "Number of Pages"
  ,"My Rating", "Bookshelves", "Exclusive Shelf"
  ,"Original Publication Year", "Book Id"
]].fillna(0).astype({
  "Number of Pages": "int32",
  "Original Publication Year": "int32"
}).sort_values(
  ["Original Publication Year", "Author l-f", "Title"]
)
book_ids = list(books_df["Book Id"])
print("book ids", book_ids)

n_books 509
columns ['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors', 'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding', 'Number of Pages', 'Year Published', 'Original Publication Year', 'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions', 'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes', 'Read Count', 'Owned Copies']
book ids [43628062, 32705383, 31863150, 62475608, 71943901, 52729898, 34298280, 55618649, 29277153, 48581590, 60673580, 204663454, 36102065, 39284489, 49695782, 58394270, 30897529, 17901517, 108014703, 36296796, 36071, 40168456, 123394897, 59957437, 42377801, 30966035, 29739642, 52376339, 30200441, 39080433, 34428125, 63365244, 42632229, 51211739, 53547613, 37509707, 30805593, 39392565, 38743103, 60712215, 39800235, 54831417, 46232004, 57057876, 42747162, 180409719, 40235580, 37794491, 58152078, 81922, 2272527, 343203, 10783088, 29502358, 49994260, 42132550, 54555961, 705418, 30295016, 63327518, 75666581,

In [33]:
len(books_df[books_df["Exclusive Shelf"]=="to-read"])

345

In [32]:
url = "https://www.goodreads.com/tooltips?"
for bn, bid in enumerate(book_ids):
  if not (bn+1)%20:
    url += "use_wtr_tooltip=true\n\nhttps://www.goodreads.com/tooltips?"
  url += "resources%5BBook."+str(bid)+"%5D%5Btype%5D=Book&"
url_f = open("url.txt", "w+", encoding="utf-8")
url_f.write(url)
url_f.close()
books_df.head()

,Author l-f,Title,Read Count,Average Rating,Number of Pages,My Rating,Bookshelves,Exclusive Shelf,Original Publication Year,Book Id
336,"Abbott, Devin",Fullstack React Native: Create beautiful mobil...,0,4.15,688,0,"to-read, informatics",to-read,0,43628062
348,"Accomazzo, Anthony",Fullstack React: The Complete Guide to ReactJS...,0,4.24,830,0,"to-read, informatics",to-read,0,32705383
270,"Antani, Ved",JavaScript: Functional Programming for JavaScr...,0,4.25,635,0,"to-read, javascript",to-read,0,31863150
238,"Arnell, R. Scott",Sustainable & Responsible Investing 360°: Less...,0,4.28,404,0,"financial-investing, to-read",to-read,0,62475608
113,"Arranz, Ana Monmeneu",VENDE TU CASA Y COMPRA TU LIBERTAD: Guía rápid...,0,3.71,182,0,to-read,to-read,0,71943901


## From HTML

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

path = '../data/books.html'
data = []
list_header = []
soup = BeautifulSoup(open(path),'html.parser', from_encoding="utf8")
header = soup.find_all("table")[0].find("tr")
for items in header:
    try:
        list_header.append(items.get_text())
    except:
        continue
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
raw_books_df = pd.DataFrame(data = data, columns = list_header).dropna()
books_df = raw_books_df[[
  'title\n \n\n'
  ,'author\n \n*\n\n'
  #,'isbn\n\n\n'
  #,'isbn13\n\n\n'
  #,'asin\n\n\n'
  ,'avg rating\n\n\n'
  ,'num ratings\n\n\n'
  ,'num pages\n\n\npp\n\n\n'
  ,'date pub\n\n\n'
  #,'date pub edition\n\n\n'
  #,'my rating\n\n\n\n\n\n'
  ,'shelves\n\n\n\n'
  #,'review\n\n\n\n\n'
  #,'notes\n\n\n\n\n'
  #,'comments\n\n\n\n'
  #,'votes\n\n\n\n'
  #,'# times read\n\n\n'
  ,'date started\n\n\n'
  ,'date read\n\n\n'
  ,'date added\n\n\n\n\n'
  #,'owned\n\n'
  #,'format\n\n        Kindle Edition\n        [edit]\n\n'
  #,'actions\n\n\n\nedit\n\nview\n               \n\n\nremove book\n\n\n\n'
]].rename(columns={
    "title\n \n\n":"title"
    ,"author\n \n*\n\n":"author"
    #,"isbn\n\n\n":"isbn"
    #,"isbn13\n\n\n":"isbn13"
    #,"asin\n\n\n":"asin"
    ,"avg rating\n\n\n":"avg rating"
    ,"num ratings\n\n\n":"num ratings"
    ,"num pages\n\n\npp\n\n\n":"num pages"
    ,"date pub\n\n\n":"date pub"
    #,"date pub edition\n\n\n":"date pub edition"
    #,"my rating\n\n\n\n\n\n":"my rating"
    ,"shelves\n\n\n\n":"shelves"
    #,"review\n\n\n\n\n":"review"
    #,"notes\n\n\n\n\n":"notes"
    #,"comments\n\n\n\n":"comments"
    #,"votes\n\n\n\n":"votes"
    #,"# times read\n\n\n":"# times read"
    ,"date started\n\n\n":"date started"
    ,"date read\n\n\n":"date read"
    ,"date added\n\n\n\n\n":"date added"
    #,"owned\n\n":"owned"
    #,"format\n\n        Kindle Edition\n        [edit]\n\n":"format"
    #,"actions\n\n\n\nedit\n\nview\n               \n\n\nremove book\n\n\n\n":"actions"
})
#for col in books_df.columns:
#  if is_string_dtype(f[col]):
#    f[col] = f[col].str.replace("[^-$\w\d <>=\"\/\[\]:\.#,?%!*'\(\);&\+áéíóúñèößºª]+", "")
books_df = books_df.replace({"[^-$A-Za-z\d <>=\"\/\[\]:\.#,?%!*'\(\);&\+áéíóúñèößºª]+": ""}, regex=True)
books_df = books_df.replace({
  "title ": "", "author ": "", "avg rating ": "", "num ratings ": "", "num pages ": "", " pp": ""
  ,"date pub ": "", "shelves": "", "\[edit\]": "", "date started": "", "date read": ""
  ,"date added ": "", "                ": " ", "        ": " ", "    ": " "
  ,"   ": " ", "  ": " "
}, regex=True)
books_df["num ratings"] = books_df["num ratings"].replace({",": ""}, regex=True)
books_df["num pages"] = books_df["num pages"].replace({",": "", "unknown": "200"}, regex=True)
books_df["date pub"] = books_df["date pub"].replace({"not set": "", "unknown": ""}, regex=True)
books_df["date started"] = books_df["date started"].replace({"not set": ""}, regex=True)
books_df["date read"] = books_df["date read"].replace({"not set": ""}, regex=True)
books_df["date added"] = books_df["date added"].replace({"not set": ""}, regex=True)
books_df = books_df.astype({
  "avg rating": "float32",
  "num ratings": "int32",
  "num pages": "int32"
})
date_pub_fixed = pd.to_datetime(books_df["date pub"],format='mixed',errors='coerce') #format='%b %d, %Y'
date_pub_fixed = date_pub_fixed.fillna(pd.to_datetime(books_df["date pub"],format='%Y',errors='coerce'))
books_df['date_pub'] = date_pub_fixed.fillna(datetime(1900, 1, 1, 0))
del books_df["date pub"]
books_df["date_started"] = pd.to_datetime(books_df["date started"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date started"]
books_df["date_read"] = pd.to_datetime(books_df["date read"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date read"]
books_df["date_added"] = pd.to_datetime(books_df["date added"],format='mixed',errors='coerce').fillna(datetime(1900, 1, 1, 0))
del books_df["date added"]
books_df["darks"] = books_df[[
  "avg rating", "num ratings", "num pages"
]].apply(
  lambda r: max(r.iloc[0]**2 + 2*np.log10(r.iloc[1]+1) - 2*np.log10(r.iloc[2]+1),0) # avg>pages>reads
  # max(np.log10((r.iloc[0]*r.iloc[1])+1) - np.log10(r.iloc[2]+1),0) strategy weighting more pages>reads>avg
  , axis=1
)
books_df = books_df.sort_values("darks", ascending=[False])
books_df.to_csv('../data/books_df.csv')
filtered_books_df = books_df[
  ~books_df["shelves"].str.contains("novels") &
  ~books_df["shelves"].str.contains("missclick")
].reset_index()
filtered_books_df.index.name='index'
filtered_books_df.to_excel("../data/filtered_books_1_df.xlsx")
filtered_books_df

In [1]:
filtered_books_df

NameError: name 'filtered_books_df' is not defined

In [161]:
filtered_books_df.dtypes

title                   object
author                  object
avg rating             float32
num ratings              int32
num pages                int32
shelves                 object
date_pub        datetime64[ns]
date_started    datetime64[ns]
date_read       datetime64[ns]
date_added      datetime64[ns]
darks                  float64
dtype: object

In [152]:
filtered_books_df[
  books_df["shelves"].str.contains("read-next-shortlist") |
  books_df["shelves"].str.contains("currently-reading")
]
filtered_books_df

C:\Users\dark_\AppData\Local\Temp\ipykernel_5928\2200323226.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_books_df[


,title,author,avg rating,num ratings,num pages,shelves,date started,date read,date added,date_pub,darks
39,"Unbroken: A World War II Story of Survival, R...","Hillenbrand, Laura",4.38,954708,475,"read-next-shortlist, history","Aug 16, 2024",not set,"Apr 19, 2022",2010-11-16,25.788929
33,The Last Lecture,"Pausch, Randy",4.26,349713,206,"read-next-shortlist, top-reference-books","Aug 16, 2024",not set,"Sep 04, 2017",2008-04-08,24.603087
70,The Psychology of Money,"Housel, Morgan",4.30,205232,252,"read-next-shortlist, psychology","Aug 16, 2024",not set,"May 30, 2022",2020-01-01,24.308255
22,"Foundation (Foundation, #1)","Asimov, Isaac",4.17,550325,244,"read-next-shortlist, novels","Aug 16, 2024",not set,"Nov 29, 2020",1951-08-30,24.091809
38,The Big Short: Inside the Doomsday Machine,"Lewis, Michael",4.30,162351,264,"read-next-shortlist, financial-investing","Aug 16, 2024",not set,"Sep 15, 2022",2010-03-15,24.064425
6,The 7 Habits of Highly Effective People: Powe...,"Covey, Stephen R.",4.16,756470,372,"read-next-shortlist, psychology, top-reference...","Aug 10, 2024",not set,"Sep 04, 2017",1994-03-01,23.919766
23,A Short History of Nearly Everything,"Bryson, Bill",4.22,393186,544,"read-next-shortlist, history","Aug 16, 2024",not set,"Sep 12, 2020",2003-01-01,23.524804
60,The Simple Path to Wealth: Your road map to f...,"Collins, J L",4.43,24775,288,"read-next-shortlist, financial-investing, psyc...","Nov 19, 2023",not set,"Nov 19, 2023",2016-06-17,23.491165
54,Homo Deus: A History of Tomorrow,"Harari, Yuval Noah",4.20,257313,450,"read-next-shortlist, history","Aug 10, 2024",not set,"May 30, 2022",2015-01-01,23.152572
40,The Lean Startup (Hardcover),"Ries, Eric*",4.11,335245,299,"read-next-shortlist, top-reference-books","Aug 10, 2024",not set,"May 30, 2022",2011-01-01,22.988586
